<a href="https://colab.research.google.com/github/jc890/python/blob/master/opencv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
OpenCV Video Analysis Test Module
---------------------------------
- Extracts frames from a video for a given time window
- Computes motion intensity using frame differencing
- Includes a built-in test for validation (Colab friendly)

Author: <Your Name>
"""

import cv2
import numpy as np
import os

# =========================
# CONFIGURATION
# =========================
FRAME_SIZE = 172
MAX_FRAMES = 50


# =========================
# FRAME EXTRACTION
# =========================
def extract_frames(video_path, start_sec, end_sec):
    """
    Extract frames from a video between start_sec and end_sec.

    Returns:
        frames (np.ndarray): Shape (T, H, W, 3)
    """

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("ERROR: Could not open video.")
        return None

    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        print("ERROR: FPS not detected.")
        return None

    start_frame = int(start_sec * fps)
    end_frame = int(end_sec * fps)

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    frames = []

    while cap.isOpened() and len(frames) < MAX_FRAMES:
        ret, frame = cap.read()
        current_frame = cap.get(cv2.CAP_PROP_POS_FRAMES)

        if not ret or current_frame > end_frame:
            break

        frame = cv2.resize(frame, (FRAME_SIZE, FRAME_SIZE))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)

    cap.release()

    if len(frames) == 0:
        return None

    return np.array(frames, dtype=np.float32)


# =========================
# MOTION INTENSITY ANALYSIS
# =========================
def compute_motion_intensity(frames):
    """
    Compute average motion intensity using frame differencing.

    Returns:
        motion_score (float)
    """

    if frames is None or len(frames) < 2:
        return 0.0

    motion_values = []

    for i in range(1, len(frames)):
        prev_gray = cv2.cvtColor(frames[i - 1].astype(np.uint8), cv2.COLOR_RGB2GRAY)
        curr_gray = cv2.cvtColor(frames[i].astype(np.uint8), cv2.COLOR_RGB2GRAY)

        diff = cv2.absdiff(prev_gray, curr_gray)
        motion = np.mean(diff)
        motion_values.append(motion)

    return float(np.mean(motion_values))


# =========================
# TEST BLOCK (RUNS IN COLAB)
# =========================
def run_test():
    """
    Test the OpenCV pipeline on a local video file.

    Instructions for Colab:
    1. Upload a short MP4 video
    2. Update VIDEO_PATH
    3. Run this script
    """

    VIDEO_PATH = "/content/test_video.mp4"

    if not os.path.exists(VIDEO_PATH):
        print(f"ERROR: {VIDEO_PATH} not found.")
        print("Upload a video named 'test_video.mp4' to the working directory.")
        return

    print("Running OpenCV analysis test...")
    print(f"Video: {VIDEO_PATH}")

    start_sec = 5
    end_sec = 15

    frames = extract_frames(VIDEO_PATH, start_sec, end_sec)

    if frames is None:
        print("Frame extraction failed.")
        return

    print(f"Frames extracted: {frames.shape}")
    print(f"Frame dtype: {frames.dtype}")

    motion_score = compute_motion_intensity(frames)
    print(f"Motion Intensity Score: {motion_score:.2f}")

    if motion_score > 15:
        print("Interpretation: High visual activity detected.")
    else:
        print("Interpretation: Low to moderate visual activity detected.")

    print("OpenCV test completed successfully.")


# =========================
# ENTRY POINT
# =========================
if __name__ == "__main__":
    run_test()


Running OpenCV analysis test...
Video: /content/test_video.mp4
Frames extracted: (50, 172, 172, 3)
Frame dtype: float32
Motion Intensity Score: 0.44
Interpretation: Low to moderate visual activity detected.
OpenCV test completed successfully.


In [1]:
!pip install opencv-python
